In [336]:
# import libraries
import numpy as np 
import pandas as pd 
from bs4 import BeautifulSoup
import requests 
import json 
from pandas.io.json import json_normalize

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes 
import geopy.geocoders

!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [489]:
# Loading the dataset containing cities and coordinates (from Google Maps) + success score [1-3] for the places where
# there are already restaurants
df_locations = pd.read_csv('/Users/petervester/Downloads/cities_old.csv', ";")
df_locations.head

<bound method NDFrame.head of              City   Latitude   Longitude  Score
0      Copenhagen  55.676097   12.568337    3.0
1            Oslo  59.913869   10.752245    3.0
2       Stockholm  59.329324   18.068581    3.0
3       Frankfurt  50.110922    8.682127    3.0
4       Barcelona  41.385064    2.173404    1.0
5          Bilbao  43.263013   -2.934985    1.0
6        Budapest  47.497120   19.040235    2.0
7           Paris  48.856614    2.352222    1.0
8            Lyon  45.764043    4.835659    1.0
9          Berlin  52.520007   13.404954    2.0
10       New York  40.712784  -74.005941    NaN
11  San Francisco  37.774929 -122.419416    NaN
12    Los Angeles  34.052234 -118.243685    NaN
13        Chicago  41.878114  -87.629798    NaN
14    Minneapolis  44.977753  -93.265011    NaN>

In [580]:
# Map of Restaurants in Europe to check if the location is correct
europe_latitude = 50; europe_longitude = 15
map_europe = folium.Map(location = [europe_latitude, europe_longitude], zoom_start = 3.4)

# add markers to map
for lat, lng, city, score in zip(df_locations['Latitude'], df_locations['Longitude'], df_locations['City'],
                                 df_locations['Score']):
    label = '{}, {}'.format(city, score)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_europe)  

map_europe

In [491]:
# Map of Possible new Restaurants in American to check if the location is correct
usa_latitude = 40; usa_longitude = -100
map_usa = folium.Map(location = [usa_latitude, usa_longitude], zoom_start = 4)

# add markers to map
for lat, lng, city, score in zip(df_locations['Latitude'], df_locations['Longitude'], df_locations['City'],
                                 df_locations['Score']):
    label = '{}, {}'.format(city, score)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_usa)  

map_usa

In [492]:
# function to get information of restaurants in 1000 m radius of city center
def foursquare_crawler (cities, lat_list, lng_list, LIMIT = 1000, radius = 1000):
    result_ds = []
    counter = 0
    for city, lat, lng in zip(cities, lat_list, lng_list):
         
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId=4d4b7105d754a06374d81259'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, 
            lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        tmp_dict = {}
        tmp_dict['Cities'] = city; tmp_dict['Latitude'] = lat; tmp_dict['Longitude'] = lng; tmp_dict['Results'] = results;
        result_ds.append(tmp_dict)
        counter += 1
        print('Obtained for City {}'.format(city))
    return result_ds;

In [493]:
# @hiddel_cell
CLIENT_ID = '0MJA3NYYG3U2ZY1LTZN2OYEHS3Y3WVSON2GBSO3IL4EDYVIR' # your Foursquare ID
CLIENT_SECRET = 'WGWSAF2TKVUQPE3PD0N3EOITFVBY5EYP1VCZI3BMUG0ROUS5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [494]:
# get information about restaurants in all the city centers
foursquare_dataset = foursquare_crawler(list(df_euro['City']),
                                                   list(df_euro['Latitude']),
                                                   list(df_euro['Longitude']),)

Obtained for City Copenhagen
Obtained for City Oslo
Obtained for City Stockholm
Obtained for City Frankfurt
Obtained for City Barcelona
Obtained for City Bilbao
Obtained for City Budapest
Obtained for City Paris
Obtained for City Lyon
Obtained for City Berlin
Obtained for City New York
Obtained for City San Francisco
Obtained for City Los Angeles
Obtained for City Chicago
Obtained for City Minneapolis


In [495]:
# Extract the Restaurant and Restaurant Categories

def get_venue_dataset(foursquare_dataset):
    result_df = pd.DataFrame(columns = ['Cities', 'Latitude', 'Longitude', 'Venue', 'Venue Category'])
    
    for neigh_dict in foursquare_dataset:
        postal_code = neigh_dict['Cities'];
        lat = neigh_dict['Latitude']; lng = neigh_dict['Longitude']
        print('Number of Venues in "{}":'.format(postal_code))
        print(len(neigh_dict['Results']))
        
        for venue_dict in neigh_dict['Results']:
            name = venue_dict['venue']['name']
            cat =  venue_dict['venue']['categories'][0]['name']
            
            result_df = result_df.append({'Cities': postal_code,
                              'Latitude': lat, 'Longitude':lng,
                              'Venue': name, 
                              'Venue Category': cat}, ignore_index = True)
    
    return(result_df)

In [496]:
df_venue = get_venue_dataset(foursquare_dataset)

Number of Venues in "Copenhagen":
100
Number of Venues in "Oslo":
100
Number of Venues in "Stockholm":
100
Number of Venues in "Frankfurt":
100
Number of Venues in "Barcelona":
100
Number of Venues in "Bilbao":
100
Number of Venues in "Budapest":
100
Number of Venues in "Paris":
100
Number of Venues in "Lyon":
100
Number of Venues in "Berlin":
100
Number of Venues in "New York":
100
Number of Venues in "San Francisco":
100
Number of Venues in "Los Angeles":
100
Number of Venues in "Chicago":
100
Number of Venues in "Minneapolis":
100


In [497]:
df_venue.head()

,Cities,Latitude,Longitude,Venue,Venue Category
0,Copenhagen,55.676097,12.568337,Lagkagehuset,Bakery
1,Copenhagen,55.676097,12.568337,The Living Room,Café
2,Copenhagen,55.676097,12.568337,Halifax,Burger Joint
3,Copenhagen,55.676097,12.568337,Väkst,Scandinavian Restaurant
4,Copenhagen,55.676097,12.568337,Next Door Cafe,Breakfast Spot


In [498]:
print('List of different categories:')
categories = list(df_venue['Venue Category'].unique())
categories

List of different categories:


['Bakery',
 'Café',
 'Burger Joint',
 'Scandinavian Restaurant',
 'Breakfast Spot',
 'Steakhouse',
 'French Restaurant',
 'Modern European Restaurant',
 'Food Truck',
 'Sandwich Place',
 'Italian Restaurant',
 'Fast Food Restaurant',
 'Mediterranean Restaurant',
 'American Restaurant',
 'Asian Restaurant',
 'Sushi Restaurant',
 'Indian Restaurant',
 'Australian Restaurant',
 'Restaurant',
 'Mexican Restaurant',
 'Bagel Shop',
 'Vietnamese Restaurant',
 'Vegetarian / Vegan Restaurant',
 'Thai Restaurant',
 'Japanese Restaurant',
 'Hot Dog Joint',
 'Food Court',
 'Seafood Restaurant',
 'Pizza Place',
 'Creperie',
 'Tapas Restaurant',
 'Taco Place',
 'Chinese Restaurant',
 'South Indian Restaurant',
 'Ramen Restaurant',
 'Peruvian Restaurant',
 'Burrito Place',
 'Falafel Restaurant',
 'Salad Place',
 'Middle Eastern Restaurant',
 'Gastropub',
 'Diner',
 'Spanish Restaurant',
 'Dumpling Restaurant',
 'Bistro',
 'Hungarian Restaurant',
 'Soup Place',
 'Comfort Food Restaurant',
 'Taverna',


In [499]:
imp_features = [
 'Cities',
 'Burger Joint',
 'Scandinavian Restaurant',
 'Steakhouse',
 'French Restaurant',
 'Italian Restaurant',
 'Fast Food Restaurant',
 'American Restaurant',
 'Asian Restaurant',
 'Sushi Restaurant',
 'Indian Restaurant',
 'Mexican Restaurant',
 'Vietnamese Restaurant',
 'Vegetarian / Vegan Restaurant',
 'Thai Restaurant',
 'Japanese Restaurant',
 'Hot Dog Joint',
 'Seafood Restaurant',
 'Pizza Place',
 'Chinese Restaurant',
 'Spanish Restaurant',
 'Greek Restaurant',
 'Turkish Restaurant',
 'African Restaurant',
 'Korean Restaurant',
 'Brazilian Restaurant',
]

In [582]:
restaurants_onehot = pd.get_dummies(data = df_venue, drop_first  = False, 
                              prefix = "", prefix_sep = "", columns = ['Venue Category'])
restaurants_onehot.head()


,Cities,Latitude,Longitude,Venue,African Restaurant,Alsatian Restaurant,American Restaurant,Apple Wine Pub,Arepa Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Auvergne Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,Brazilian Restaurant,Breakfast Spot,Burger Joint,Burgundian Restaurant,Burmese Restaurant,Burrito Place,Cafeteria,Café,Cajun / Creole Restaurant,Chinese Restaurant,Churrascaria,Cigkofte Place,Comfort Food Restaurant,Corsican Restaurant,Creperie,Cuban Restaurant,Currywurst Joint,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fondue Restaurant,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Grilled Meat Restaurant,Hawaiian Restaurant,Hot Dog Joint,Hungarian Restaurant,Indian Restaurant,Indonesian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Lyonese Bouchon,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,Paella Restaurant,Peking Duck Restaurant,Peruvian Restaurant,Pet Café,Pizza Place,Poke Place,Polish Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Russian Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Snack Place,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spanish Restaurant,Steakhouse,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Taverna,Thai Restaurant,Theme Restaurant,Trattoria/Osteria,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint,Yoshoku Restaurant
0,Copenhagen,55.676097,12.568337,Lagkagehuset,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Copenhagen,55.676097,12.568337,The Living Room,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Copenhagen,55.676097,12.568337,Halifax,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Copenhagen,55.676097,12.568337,Väkst,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Copenhagen,55.676097,12.568337,Next Door Cafe,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [583]:
restaurants_onehot = restaurants_onehot[imp_features].groupby('Cities').sum()
restaurants_onehot

,Burger Joint,Scandinavian Restaurant,Steakhouse,French Restaurant,Italian Restaurant,Fast Food Restaurant,American Restaurant,Asian Restaurant,Sushi Restaurant,Indian Restaurant,Mexican Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Japanese Restaurant,Hot Dog Joint,Seafood Restaurant,Pizza Place,Chinese Restaurant,Spanish Restaurant,Greek Restaurant,Turkish Restaurant,African Restaurant,Korean Restaurant,Brazilian Restaurant
Cities,,,,,,,,,,,,,,,,,,,,,,,,,
Barcelona,4,0,1,0,6,0,1,2,0,0,2,1,8,0,5,1,1,4,0,11,1,0,0,0,0
Berlin,3,0,1,0,8,0,0,4,1,0,0,11,2,3,3,0,0,0,0,1,0,0,0,2,0
Bilbao,2,0,2,0,3,1,0,3,1,0,1,0,2,0,3,1,6,2,0,15,0,0,0,0,0
Budapest,2,0,3,3,13,0,1,0,1,2,1,2,1,1,3,0,0,1,0,0,0,0,0,0,0
Chicago,4,0,1,2,11,0,4,3,0,1,1,1,3,0,1,1,1,7,0,0,0,0,1,0,0
Copenhagen,5,18,3,7,3,1,2,2,2,2,1,1,2,2,3,1,1,2,1,0,0,0,0,0,0
Frankfurt,3,0,3,2,10,0,0,2,2,1,1,1,0,6,5,0,1,2,0,0,1,2,1,0,0
Los Angeles,0,0,1,3,6,0,1,0,10,1,9,0,3,1,14,0,3,2,0,0,0,0,0,1,0
Lyon,4,1,2,29,5,4,1,0,2,2,0,0,0,2,5,0,2,2,1,0,1,0,0,0,0


In [565]:
restaurants_onehot['Total Restaurants'] = restaurants_onehot.sum(axis = 1)
scarborough_onehot

,Burger Joint,Scandinavian Restaurant,Steakhouse,French Restaurant,Italian Restaurant,Fast Food Restaurant,American Restaurant,Asian Restaurant,Sushi Restaurant,Indian Restaurant,Mexican Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Japanese Restaurant,Hot Dog Joint,Seafood Restaurant,Pizza Place,Chinese Restaurant,Spanish Restaurant,Greek Restaurant,Turkish Restaurant,African Restaurant,Korean Restaurant,Brazilian Restaurant,Total Restaurants
Cities,,,,,,,,,,,,,,,,,,,,,,,,,,
Barcelona,4,0,1,0,6,0,1,2,0,0,2,1,8,0,5,1,1,4,0,11,1,0,0,0,0,96
Berlin,3,0,1,0,8,0,0,4,1,0,0,11,2,3,3,0,0,0,0,1,0,0,0,2,0,78
Bilbao,2,0,2,0,3,1,0,3,1,0,1,0,2,0,3,1,6,2,0,15,0,0,0,0,0,84
Budapest,2,0,3,3,13,0,1,0,1,2,1,2,1,1,3,0,0,1,0,0,0,0,0,0,0,68
Chicago,4,0,1,2,11,0,4,3,0,1,1,1,3,0,1,1,1,7,0,0,0,0,1,0,0,84
Copenhagen,5,18,3,7,3,1,2,2,2,2,1,1,2,2,3,1,1,2,1,0,0,0,0,0,0,118
Frankfurt,3,0,3,2,10,0,0,2,2,1,1,1,0,6,5,0,1,2,0,0,1,2,1,0,0,86
Los Angeles,0,0,1,3,6,0,1,0,10,1,9,0,3,1,14,0,3,2,0,0,0,0,0,1,0,110
Lyon,4,1,2,29,5,4,1,0,2,2,0,0,0,2,5,0,2,2,1,0,1,0,0,0,0,126


In [566]:
from sklearn.cluster import KMeans

# run k-means clustering in 5 groups
kmeans = KMeans(n_clusters = 6, random_state = 0).fit(restaurants_onehot)

In [567]:
# investigate centers
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = restaurants_onehot.columns
means_df.index = ['1','2','3','4','5', '6']
means_df['Total Sum'] = means_df.sum(axis = 1)
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Burger Joint,Scandinavian Restaurant,Steakhouse,French Restaurant,Italian Restaurant,Fast Food Restaurant,American Restaurant,Asian Restaurant,Sushi Restaurant,Indian Restaurant,Mexican Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Japanese Restaurant,Hot Dog Joint,Seafood Restaurant,Pizza Place,Chinese Restaurant,Spanish Restaurant,Greek Restaurant,Turkish Restaurant,African Restaurant,Korean Restaurant,Brazilian Restaurant,Total Restaurants,Total Sum
1,4.500000,1.500000,1.500000,32.000000,2.500000,2.0,1.000000,0.000000,1.500000,1.000000,0.500000,0.000000,0.500000,1.500000,3.500000,0.500000,2.000000,2.000000,1.500000,0.000000e+00,0.500000,0.0,5.000000e-01,0.000000,0.000000,60.500000,121.000000
3,4.666667,17.333333,1.666667,4.333333,3.333333,1.0,0.666667,3.666667,2.666667,3.333333,0.666667,2.000000,1.333333,1.666667,2.333333,0.333333,2.666667,2.666667,1.000000,3.333333e-01,0.000000,0.0,-2.775558e-17,0.000000,0.000000,57.666667,115.333333
5,0.000000,0.000000,1.000000,3.000000,6.000000,0.0,1.000000,0.000000,10.000000,1.000000,9.000000,0.000000,3.000000,1.000000,14.000000,0.000000,3.000000,2.000000,0.000000,0.000000e+00,0.000000,0.0,0.000000e+00,1.000000,0.000000,55.000000,110.000000
2,2.333333,1.666667,2.000000,4.000000,5.333333,1.0,5.000000,2.666667,4.333333,1.666667,3.000000,0.666667,1.333333,1.333333,1.333333,0.000000,0.666667,4.666667,2.333333,2.220446e-16,0.666667,0.0,-2.775558e-17,0.666667,0.333333,47.000000,94.000000
4,3.000000,0.000000,1.500000,0.000000,4.500000,0.5,0.500000,2.500000,0.500000,0.000000,1.500000,0.500000,5.000000,0.000000,4.000000,1.000000,3.500000,3.000000,0.000000,1.300000e+01,0.500000,0.0,0.000000e+00,0.000000,0.000000,45.000000,90.000000
6,3.000000,0.000000,2.000000,1.750000,10.500000,0.0,1.250000,2.250000,1.000000,1.000000,0.750000,3.750000,1.500000,2.500000,3.000000,0.250000,0.500000,2.500000,0.000000,2.500000e-01,0.250000,0.5,5.000000e-01,0.500000,0.000000,39.500000,79.000000


In [568]:
# investigate clusters
rest_groups = pd.DataFrame([scarborough_onehot.index, 1 + kmeans.labels_]).T
rest_groups.columns = ['City', 'Group']
rest_groups

,City,Group
0,Barcelona,4
1,Berlin,6
2,Bilbao,4
3,Budapest,6
4,Chicago,6
5,Copenhagen,3
6,Frankfurt,6
7,Los Angeles,5
8,Lyon,1
9,Minneapolis,2


In [569]:
rest_groups[rest_groups['Group'] == 1] # group 1 is all the french restaurants (avg. score = 1)

,City,Group
8,Lyon,1
12,Paris,1


In [570]:
rest_groups[rest_groups['Group'] == 2] # group 2 is mixed of german/hungarian/american restaurants (avg. score = 2.33)

,City,Group
9,Minneapolis,2
10,New York,2
13,San Francisco,2


In [571]:
rest_groups[rest_groups['Group'] == 3] # group 3 is all the scandinavian restaurants (avg. score = 3)

,City,Group
5,Copenhagen,3
11,Oslo,3
14,Stockholm,3


In [572]:
rest_groups[rest_groups['Group'] == 4] # group 4 is all the spanish restaurants (avg. score = 1)

,City,Group
0,Barcelona,4
2,Bilbao,4


In [573]:
rest_groups[rest_groups['Group'] == 5] # Los Angeles is in its own group 

,City,Group
7,Los Angeles,5


In [574]:
## Recommender System for which city to open restaurant in the states!!

In [575]:
rest_groups[rest_groups['Group'] == 6] # Los Angeles is in its own group 

,City,Group
1,Berlin,6
3,Budapest,6
4,Chicago,6
6,Frankfurt,6


In [511]:
df_locations.head

<bound method NDFrame.head of              City   Latitude   Longitude  Score
0      Copenhagen  55.676097   12.568337    3.0
1            Oslo  59.913869   10.752245    3.0
2       Stockholm  59.329324   18.068581    3.0
3       Frankfurt  50.110922    8.682127    3.0
4       Barcelona  41.385064    2.173404    1.0
5          Bilbao  43.263013   -2.934985    1.0
6        Budapest  47.497120   19.040235    2.0
7           Paris  48.856614    2.352222    1.0
8            Lyon  45.764043    4.835659    1.0
9          Berlin  52.520007   13.404954    2.0
10       New York  40.712784  -74.005941    NaN
11  San Francisco  37.774929 -122.419416    NaN
12    Los Angeles  34.052234 -118.243685    NaN
13        Chicago  41.878114  -87.629798    NaN
14    Minneapolis  44.977753  -93.265011    NaN>

In [512]:
restaurants_onehot['City'] = restaurants_onehot.index
restaurants_onehot

,Burger Joint,Scandinavian Restaurant,Steakhouse,French Restaurant,Italian Restaurant,Fast Food Restaurant,American Restaurant,Asian Restaurant,Sushi Restaurant,Indian Restaurant,Mexican Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Japanese Restaurant,Hot Dog Joint,Seafood Restaurant,Pizza Place,Chinese Restaurant,Spanish Restaurant,Greek Restaurant,Turkish Restaurant,African Restaurant,Korean Restaurant,Brazilian Restaurant,Total Restaurants,City
Cities,,,,,,,,,,,,,,,,,,,,,,,,,,,
Barcelona,4,0,1,0,6,0,1,2,0,0,2,1,8,0,5,1,1,4,0,11,1,0,0,0,0,48,Barcelona
Berlin,3,0,1,0,8,0,0,4,1,0,0,11,2,3,3,0,0,0,0,1,0,0,0,2,0,39,Berlin
Bilbao,2,0,2,0,3,1,0,3,1,0,1,0,2,0,3,1,6,2,0,15,0,0,0,0,0,42,Bilbao
Budapest,2,0,3,3,13,0,1,0,1,2,1,2,1,1,3,0,0,1,0,0,0,0,0,0,0,34,Budapest
Chicago,4,0,1,2,11,0,4,3,0,1,1,1,3,0,1,1,1,7,0,0,0,0,1,0,0,42,Chicago
Copenhagen,5,18,3,7,3,1,2,2,2,2,1,1,2,2,3,1,1,2,1,0,0,0,0,0,0,59,Copenhagen
Frankfurt,3,0,3,2,10,0,0,2,2,1,1,1,0,6,5,0,1,2,0,0,1,2,1,0,0,43,Frankfurt
Los Angeles,0,0,1,3,6,0,1,0,10,1,9,0,3,1,14,0,3,2,0,0,0,0,0,1,0,55,Los Angeles
Lyon,4,1,2,29,5,4,1,0,2,2,0,0,0,2,5,0,2,2,1,0,1,0,0,0,0,63,Lyon


In [513]:
df_locations["Cities"] = df_locations["City"]
df_classifier = pd.merge(df_locations, restaurants_onehot, on='City', how='outer')
df_cl = df_classifier.drop(["Latitude", "Longitude", "Cities"], axis = 1)
df_cll = df_cl.set_index('City')
df_cll

,Score,Burger Joint,Scandinavian Restaurant,Steakhouse,French Restaurant,Italian Restaurant,Fast Food Restaurant,American Restaurant,Asian Restaurant,Sushi Restaurant,Indian Restaurant,Mexican Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Japanese Restaurant,Hot Dog Joint,Seafood Restaurant,Pizza Place,Chinese Restaurant,Spanish Restaurant,Greek Restaurant,Turkish Restaurant,African Restaurant,Korean Restaurant,Brazilian Restaurant,Total Restaurants
City,,,,,,,,,,,,,,,,,,,,,,,,,,,
Copenhagen,3.0,5,18,3,7,3,1,2,2,2,2,1,1,2,2,3,1,1,2,1,0,0,0,0,0,0,59
Oslo,3.0,5,7,1,5,2,2,0,4,3,8,0,4,2,3,4,0,1,5,2,1,0,0,0,0,0,59
Stockholm,3.0,4,27,1,1,5,0,0,5,3,0,1,1,0,0,0,0,6,1,0,0,0,0,0,0,0,55
Frankfurt,3.0,3,0,3,2,10,0,0,2,2,1,1,1,0,6,5,0,1,2,0,0,1,2,1,0,0,43
Barcelona,1.0,4,0,1,0,6,0,1,2,0,0,2,1,8,0,5,1,1,4,0,11,1,0,0,0,0,48
Bilbao,1.0,2,0,2,0,3,1,0,3,1,0,1,0,2,0,3,1,6,2,0,15,0,0,0,0,0,42
Budapest,2.0,2,0,3,3,13,0,1,0,1,2,1,2,1,1,3,0,0,1,0,0,0,0,0,0,0,34
Paris,1.0,5,2,1,35,0,0,1,0,1,0,1,0,1,1,2,1,2,2,2,0,0,0,1,0,0,58
Lyon,1.0,4,1,2,29,5,4,1,0,2,2,0,0,0,2,5,0,2,2,1,0,1,0,0,0,0,63


In [514]:
X_train = df_cll[0:10].drop(['Score'], axis = 1)
X_test = df_cll[10:15].drop(['Score'], axis = 1)
Y_train = df_cll['Score'][0:10]
X_train

,Burger Joint,Scandinavian Restaurant,Steakhouse,French Restaurant,Italian Restaurant,Fast Food Restaurant,American Restaurant,Asian Restaurant,Sushi Restaurant,Indian Restaurant,Mexican Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Japanese Restaurant,Hot Dog Joint,Seafood Restaurant,Pizza Place,Chinese Restaurant,Spanish Restaurant,Greek Restaurant,Turkish Restaurant,African Restaurant,Korean Restaurant,Brazilian Restaurant,Total Restaurants
City,,,,,,,,,,,,,,,,,,,,,,,,,,
Copenhagen,5,18,3,7,3,1,2,2,2,2,1,1,2,2,3,1,1,2,1,0,0,0,0,0,0,59
Oslo,5,7,1,5,2,2,0,4,3,8,0,4,2,3,4,0,1,5,2,1,0,0,0,0,0,59
Stockholm,4,27,1,1,5,0,0,5,3,0,1,1,0,0,0,0,6,1,0,0,0,0,0,0,0,55
Frankfurt,3,0,3,2,10,0,0,2,2,1,1,1,0,6,5,0,1,2,0,0,1,2,1,0,0,43
Barcelona,4,0,1,0,6,0,1,2,0,0,2,1,8,0,5,1,1,4,0,11,1,0,0,0,0,48
Bilbao,2,0,2,0,3,1,0,3,1,0,1,0,2,0,3,1,6,2,0,15,0,0,0,0,0,42
Budapest,2,0,3,3,13,0,1,0,1,2,1,2,1,1,3,0,0,1,0,0,0,0,0,0,0,34
Paris,5,2,1,35,0,0,1,0,1,0,1,0,1,1,2,1,2,2,2,0,0,0,1,0,0,58
Lyon,4,1,2,29,5,4,1,0,2,2,0,0,0,2,5,0,2,2,1,0,1,0,0,0,0,63


In [593]:
# classify with neighest neighbour method
from sklearn.neighbors import KNeighborsRegressor
from sklearn import preprocessing

svclassifier = KNeighborsRegressor(n_neighbors=5)
scaler = preprocessing.StandardScaler().fit(X_train)
scaler.transform(X_train) 

svclassifier.fit(X_train, Y_train)  

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [594]:
scaler.transform(X_test) 
y_pred = svclassifier.predict(X_test)  
y_pred

array([2.2, 1.8, 2.2, 1.8, 2. ])

In [595]:
# show results
results = pd.DataFrame()
results["Score Pred."] = y_pred
results["City"] = df_cll.index[10:15]
results

,Score Pred.,City
0,2.2,New York
1,1.8,San Francisco
2,2.2,Los Angeles
3,1.8,Chicago
4,2.0,Minneapolis
